In [5]:
!nvidia-smi

Sun Jul 20 15:32:11 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   39C    P0              34W / 250W |  12422MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch

# Set device (GPU if available)
DEVICE_NUM = 4
ADDITIONAL_GPU = 1

if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_NUM)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}:{DEVICE_NUM}")

INFO: Using device - cuda:4


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import numpy as np
import time

# 1. 기본 설정 및 데이터 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# C-proxy 계산을 위해 셔플하지 않은 로더 사용
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)

# 2. 모델 정의
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# 3. C-proxy 계산을 위한 훈련 및 평가
epochs = 20 # 예시 에포크
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# 각 훈련 데이터 포인트의 confidence를 저장할 배열
# 논문에서는 훈련 중 모든 epoch에서 confidence를 추적하고 평균냄
confidence_history = np.zeros((len(train_dataset), epochs))

start_time = time.time()

for epoch in range(epochs):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # --- 각 에포크가 끝난 후 C-proxy 계산을 위한 confidence 기록 ---
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            # Softmax를 적용하여 확률(confidence) 계산
            softmax_probs = torch.softmax(outputs, dim=1)
            # 각 데이터의 정답 레이블에 대한 confidence 추출
            correct_class_confidence = softmax_probs[torch.arange(len(targets)), targets]

            # 기록
            start_index = batch_idx * train_loader.batch_size
            end_index = start_index + len(targets)
            confidence_history[start_index:end_index, epoch] = correct_class_confidence.cpu().numpy()

    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs} 완료. 소요 시간: {time.time() - start_time:.2f}초")


# 4. 최종 C-proxy 점수 계산
# 모든 에포크에 대한 confidence의 평균을 계산
c_proxy_scores = np.mean(confidence_history, axis=1)

print("\nC-proxy 계산 완료")
print(f"계산된 C-proxy 점수 (상위 5개): {c_proxy_scores[:5]}")
print(f"C-proxy 점수 배열 크기: {c_proxy_scores.shape}")

# 점수가 낮을수록 더 많이 암기되었을 가능성이 높음
# 상위 5개 암기된 데이터의 인덱스
most_memorized_indices = np.argsort(c_proxy_scores)[:5]
print(f"가장 많이 암기된 것으로 추정되는 데이터 인덱스 (C-proxy 점수가 낮은 순): {most_memorized_indices}")
print(f"해당 데이터들의 C-proxy 점수: {c_proxy_scores[most_memorized_indices]}")

Using device: cuda


100.0%


Epoch 1/20 완료. 소요 시간: 48.26초
Epoch 2/20 완료. 소요 시간: 94.91초
Epoch 3/20 완료. 소요 시간: 141.58초
Epoch 4/20 완료. 소요 시간: 188.00초
Epoch 5/20 완료. 소요 시간: 234.12초
Epoch 6/20 완료. 소요 시간: 280.58초
Epoch 7/20 완료. 소요 시간: 326.50초
Epoch 8/20 완료. 소요 시간: 372.36초
Epoch 9/20 완료. 소요 시간: 418.36초
Epoch 10/20 완료. 소요 시간: 465.33초
Epoch 11/20 완료. 소요 시간: 511.80초
Epoch 12/20 완료. 소요 시간: 558.27초
Epoch 13/20 완료. 소요 시간: 603.69초
Epoch 14/20 완료. 소요 시간: 650.38초
Epoch 15/20 완료. 소요 시간: 695.98초
Epoch 16/20 완료. 소요 시간: 742.37초
Epoch 17/20 완료. 소요 시간: 789.38초
Epoch 18/20 완료. 소요 시간: 835.40초
Epoch 19/20 완료. 소요 시간: 881.27초
Epoch 20/20 완료. 소요 시간: 926.63초

C-proxy 계산 완료!
계산된 C-proxy 점수 (상위 5개): [0.89213359 0.97150561 0.85158746 0.96729454 0.92375098]
C-proxy 점수 배열 크기: (50000,)
가장 많이 암기된 것으로 추정되는 데이터 인덱스 (C-proxy 점수가 낮은 순): [10755 22515 17022 20627 30707]
해당 데이터들의 C-proxy 점수: [0.21150554 0.22249599 0.24543833 0.29612337 0.30267277]
